## Código de una clase anterior



In [1]:
import matplotlib.pyplot as plt
import networkx as nx
from random import random
from scipy.spatial.distance import euclidean
import numpy as np

from pycliques.dominated import completely_pared_graph

def puntos_en_el_plano(n,
                       condicion=lambda punto: True,
                       int_x=[0, 1],
                       int_y=[0, 1]):
    listax = []
    listay = []
    while len(listax) < n:
        coord_x = (int_x[1]-int_x[0])*random() + int_x[0]
        coord_y = (int_y[1]-int_y[0])*random() + int_y[0]
        if condicion((coord_x, coord_y)):
            listax.append(coord_x)
            listay.append(coord_y)
    return listax, listay

def grafica_rips(listax, listay, epsilon):
    g = nx.Graph()
    n = len(listax)
    vertices = []
    aristas = []
    for i in range(n):
        vertices.append((listax[i], listay[i]))
    for i in range(n):
        for j in range(i+1, n):
            if euclidean(vertices[i], vertices[j]) <= epsilon:
                aristas.append((vertices[i], vertices[j]))
    g.add_nodes_from(vertices)
    g.add_edges_from(aristas)
    return g

def dibujo_grafica_rips(listax, listay,
                        epsilon,
                        int_x = [0, 1],
                        int_y = [0, 1]):
    plt.figure(1, figsize = (8, 8))
    g = grafica_rips(listax, listay, epsilon)
    for e in g.edges():
        u = e[0]
        v = e[1]
        plt.plot([u[0], v[0]], [u[1], v[1]], color='blue')
    plt.plot(listax, listay, 'ro')
    plt.axis([int_x[0] - 0.1,
              int_x[1] + 0.1,
              int_y[0] - 0.1,
              int_y[1] + 0.1])
    plt.gca().set_aspect('equal')
    plt.show()

In [1]:
coords_x, coords_y = puntos_en_el_plano(50)

In [1]:
dibujo_grafica_rips(coords_x, coords_y, .24)

## Mogutda



[https://github.com/stephenhky/MoguTDA](https://github.com/stephenhky/MoguTDA)



In [1]:
import mogutda

def dimension_sc(complex):
    dims = [len(f) for f in complex.face_set]
    dims.sort()
    return dims[-1] - 1

# números de betti del complejo de completas de una gráfica
def betti_numbers(graph):
    def simplify_list(bettis):
        simplified = bettis
        while len(simplified) > 0 and simplified[-1] == 0:
            del simplified[-1]
        return simplified
    the_simplices = [tuple(c) for c in nx.find_cliques(graph)]
    the_complex = mogutda.SimplicialComplex(simplices = the_simplices)
    dim = dimension_sc(the_complex)
    numbers = [the_complex.betti_number(i) for i in range(dim+1)]
    numbers[0] = numbers[0] - 1 # reduced betti number
    return simplify_list(numbers)

In [1]:
complejo1 = mogutda.SimplicialComplex(simplices=[[0,1,2], [1,3], [2,3]])

In [1]:
dimension_sc(complejo1)

In [1]:
complejo1.betti_number(0)

In [1]:
complejo1.betti_number(1)

In [1]:
complejo2 = mogutda.SimplicialComplex(simplices=[[0,1],[0,2],[1,2],[1,3],[2,3]])
dimension_sc(complejo2)

In [1]:
complejo2.betti_number(0), complejo2.betti_number(1), complejo2.betti_number(2)

## Números de Betti de los complejos de Rips



In [1]:
com = mogutda.SimplicialComplex(simplices=nx.find_cliques(grafica_rips(coords_x, coords_y, 0.23)))
com.betti_number(0), com.betti_number(1), com.betti_number(2)

Definimos una función que calcula todos los números de Betti del complejo de completas de una gráfica.



In [1]:
betti_numbers(grafica_rips(coords_x, coords_y, 0.24))

Cuando el parámetro es pequeño y la gráfica tiene muy pocas aristas, los números de Betti se pueden calcular directamente.

Cuando hay muchas aristas conviene primero simplificar la gráfica antes de calcular los números de Betti.



In [1]:
betti_numbers(completely_pared_graph(grafica_rips(coords_x, coords_y, 0.8)))

## Diagrama de números de Betti



In [1]:
def dict_of_betti_numbers(coords_x, coords_y, num_params=100, left=0, right=1):
    pars = np.linspace(left, right, num_params+1)
    return {par: betti_numbers(completely_pared_graph(grafica_rips(coords_x, coords_y, par))) for par in pars}

def max_dimension_of_dict(the_dict):
    dims = [len(the_dict[i]) for i in the_dict]
    dims.sort()
    return dims[-1]

def graph_of_all_betti_numbers(the_dict):
    def the_value(the_list, i):
        if i < len(the_list):
            return the_list[i]
        else:
            return 0
    the_dim = max_dimension_of_dict(the_dict) + 1
    fig, ax = plt.subplots()
    for dim in range(the_dim):
        components = [the_value(the_dict[i], dim) for i in the_dict]
        ax.plot(list(the_dict.keys()), components, label=r'$\beta_{}$'.format(dim))
    ax.set_xlabel('Parámetro')    
    ax.set_ylabel('Número de Betti')
    ax.legend()
    #fig.savefig("./res/{}.pdf".format(title))
    plt.show()

In [1]:
the_dict = dict_of_betti_numbers(coords_x, coords_y)

In [1]:
the_dict

In [1]:
graph_of_all_betti_numbers(the_dict)

## Datos no aleatorios



In [1]:
def dentro_de_anillo(punto, r=0.8, epsilon=0.1):
    distancia = euclidean(punto, 0)
    return distancia < r+epsilon and distancia > r-epsilon

In [1]:
coords_x, coords_y = puntos_en_el_plano(50,
                                        condicion = dentro_de_anillo,
                                        int_x = [-1, 1],
                                        int_y = [-1, 1])

In [1]:
dibujo_grafica_rips(coords_x, coords_y, .57, int_x=[-1,1], int_y=[-1, 1])

In [1]:
the_dict = dict_of_betti_numbers(coords_x, coords_y)
graph_of_all_betti_numbers(the_dict)